<a href="https://colab.research.google.com/github/mariotv3/MDL/blob/main/IntroToSeminarCases.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!python -m pip install gurobipy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 31.8 MB/s eta 0:00:00


In [ ]:
import gurobipy as gp
from gurobipy import *
import pandas as pd
import numpy as np

In [ ]:
# importing values from excel file
df = pd.read_excel('Railway services-2024.xlsx')
df = df.iloc[:-3]
indexes = df['Trip'].tolist()
demand = df['Demand(μ)'].tolist()
line = df['Line'].tolist()
demand_stdDev = df['Demand(σ)'].tolist()

Exercise 3 - Formulation 1 (Cross-sections + train types)

In [ ]:
# create model
m1 = gp.Model("first formulation ex 3")

# create variables
N = m1.addVars(2, 200, vtype=GRB.INTEGER) #N_{u,c} variables (2 types & 200 cross-sections) (0 = OC & 1 = OH)

# create coefficients
lengthRequirement = [200 if x == 400 else 300 for x in line]
costCoefficients = [260000, 210000]
costDict = {}
capacityCoefficients = [620, 420]
capacityDict = {}
lengthCoefficients = [100, 70]
lengthDict = {}
qty1Coefficients = [1, -1.25]
qty1Dict = {}
qty2Coefficients = [-1.25, 1]
qty2Dict = {}

for u in range(2):
  for c in range(200):
    costDict[(u, c)] = costCoefficients[u]
    capacityDict[(u, c)] = capacityCoefficients[u]
    lengthDict[(u, c)] = lengthCoefficients[u]
    qty1Dict[(u, c)] = qty1Coefficients[u]
    qty2Dict[(u, c)] = qty2Coefficients[u]

Restricted license - for non-production use only - expires 2025-11-24


In [ ]:
# Defining Objective Function
m1.setObjective(N.prod(costDict))

# Generating Constraints
m1.addConstrs(N.prod(capacityDict, '*', c) >= demand[c]  for c in range(200))
m1.addConstr(N.prod(qty1Dict) <= 0)
m1.addConstr(N.prod(qty2Dict) <= 0)
m1.addConstrs(N.prod(lengthDict, '*', c) <= lengthRequirement[c] for c in range(200))
m1.addConstrs(N[u, c] >= 0 for u in range(2) for c in range(200))

m1.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 802 rows, 400 columns and 2000 nonzeros
Model fingerprint: 0x2125fd01
Variable types: 0 continuous, 400 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 6e+02]
  Objective range  [2e+05, 3e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 1e+03]
Presolve removed 424 rows and 23 columns
Presolve time: 0.01s
Presolved: 378 rows, 377 columns, 1506 nonzeros
Variable types: 0 continuous, 377 integer (0 binary)
Found heuristic solution: objective 9.384000e+07

Root relaxation: objective 7.121878e+07, 282 iterations, 0.01 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0 

In [ ]:
# Solve LP relaxation
r1 = m1.relax()
r1.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 802 rows, 400 columns and 2000 nonzeros
Model fingerprint: 0x87787649
Coefficient statistics:
  Matrix range     [1e+00, 6e+02]
  Objective range  [2e+05, 3e+05]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+02, 1e+03]
Presolve removed 400 rows and 0 columns
Presolve time: 0.01s
Presolved: 402 rows, 400 columns, 1600 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   4.268188e+03   0.000000e+00      0s
     270    6.1147590e+07   0.000000e+00   0.000000e+00      0s

Solved in 270 iterations and 0.02 seconds (0.00 work units)
Optimal objective  6.114758996e+07


Exercise 3 - Formulation 2 (Compositions + Cross-sections)

In [ ]:
# Create set of compositions - denoted as a pair of values (OC, OH), where OC is the number of OC rolling stock units and OH is the number of OH rolling stock units
compositions = []
minDemand = min(demand)

for OC in range(4):
  for OH in range(5):
    if 620 * OC + 420 * OH >= min(demand) and 100 * OC + 70 * OH <= 300:
      compositions.append((OC, OH))

print(compositions)

[(0, 1), (0, 2), (0, 3), (0, 4), (1, 0), (1, 1), (1, 2), (2, 0), (2, 1), (3, 0)]


In [ ]:
# Create model
m2 = gp.Model("second formulation ex 3")

# create variables
X = m2.addVars(200, 10, vtype=GRB.BINARY) #X_{c,p} variables (200 cross-sections & 10 compositions)

# create coefficients
lengthRequirement = [200 if x == 400 else 300 for x in line]
costCoefficients = [260000, 210000]
costDict2 = {}
capacityCoefficients = [620, 420]
capacityDict2 = {}
lengthCoefficients = [100, 70]
lengthDict2 = {}
qty1Coefficients = [1, -1.25]
qty1Dict2 = {}
numOfType1Trains = {}
numOfType2Trains = {}
qty2Coefficients = [-1.25, 1]
qty2Dict2 = {}

for composition in range(len(compositions)):
  for c in range(200):
    costDict2[(c, composition)] = costCoefficients[0] * compositions[composition][0] + costCoefficients[1] * compositions[composition][1]
    capacityDict2[(c, composition)] = capacityCoefficients[0] * compositions[composition][0] + capacityCoefficients[1] * compositions[composition][1]
    lengthDict2[(c, composition)] = lengthCoefficients[0] * compositions[composition][0] + lengthCoefficients[1] * compositions[composition][1]
    qty1Dict2[(c, composition)] = qty1Coefficients[0] * compositions[composition][0] + qty1Coefficients[1] * compositions[composition][1]
    qty2Dict2[(c, composition)] = qty2Coefficients[0] * compositions[composition][0] + qty2Coefficients[1] * compositions[composition][1]

In [ ]:
# Defining Objective Function
m2.setObjective(X.prod(costDict2))

# Generating Constraints
m2.addConstrs(X.prod(capacityDict2, c, '*') >= demand[c]  for c in range(200))
m2.addConstr(X.prod(qty1Dict2) <= 0)
m2.addConstr(X.prod(qty2Dict2) <= 0)
m2.addConstrs(X.prod(lengthDict2, c, '*') <= lengthRequirement[c] for c in range(200))
m2.addConstrs(X.sum(c, '*') == 1 for c in range(200))

m2.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 602 rows, 2000 columns and 10000 nonzeros
Model fingerprint: 0x1848086d
Variable types: 0 continuous, 2000 integer (2000 binary)
Coefficient statistics:
  Matrix range     [2e-01, 2e+03]
  Objective range  [2e+05, 8e+05]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve removed 404 rows and 670 columns
Presolve time: 0.04s
Presolved: 198 rows, 1330 columns, 3949 nonzeros
Variable types: 0 continuous, 1330 integer (1329 binary)
Found heuristic solution: objective 7.524000e+07

Root relaxation: objective 7.291778e+07, 129 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node T

In [ ]:
# Solve LP relaxation
r2 = m2.relax()
r2.optimize()

Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 602 rows, 2000 columns and 10000 nonzeros
Model fingerprint: 0x97998a3d
Coefficient statistics:
  Matrix range     [2e-01, 2e+03]
  Objective range  [2e+02, 8e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+03]
Presolve time: 0.02s
Presolved: 602 rows, 2000 columns, 10000 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.2000000e+04   4.945719e+03   0.000000e+00      0s
     339    6.2616490e+04   0.000000e+00   0.000000e+00      0s

Solved in 339 iterations and 0.06 seconds (0.01 work units)
Optimal objective  6.261648954e+04


Exercise 4 - Stochastic Demand

In [ ]:
# Create set of compositions - denoted as a pair of values (OC, OH), where OC is the number of OC rolling stock units and OH is the number of OH rolling stock units
compositions = []
minDemand = min(demand)

for OC in range(4):
  for OH in range(5):
    if 620 * OC + 420 * OH >= min(demand) and 100 * OC + 70 * OH <= 300:
      compositions.append((OC, OH))

# Create stochastic demand
stochasticDemands = [[0 for days in range(250)] for c in range(200)] # stochasticDemands[c][days]

for c in range(200):
  demandMean = demand[c]
  demandStDev = demand_stdDev[c]
  for day in range(250):
    stochasticDemands[c][day] = np.random.normal(demandMean, demand_stdDev)

params = {
"WLSACCESSID": '9f6bda1d-2c0b-49c7-96a8-f172b568e9e4',
"WLSSECRET": '3db7e79a-7091-4079-9dc3-c9c1382b1c4e',
"LICENSEID": 2498504,
}
env = gp.Env(params=params)

Set parameter WLSAccessID
Set parameter WLSSecret
Set parameter LicenseID to value 2498504
Academic license 2498504 - for non-commercial use only - registered to 61___@eur.nl


In [ ]:
# Create model
m3 = gp.Model("Exercise 4 second formulation", env = env)

# Create variables
X2 = m3.addVars(200, 10, vtype=GRB.BINARY) #X_{c,p} variables (200 cross-sections & 10 compositions)
Y = m3.addVars(200, 250, vtype=GRB.BINARY) #Y_{c, d} variables (200 cross-sections & 250 days)

# Create coefficients
lengthRequirement = [200 if x == 400 else 300 for x in line]
costCoefficients = [260000, 210000]
costDict3 = {}
capacityCoefficients = [620, 420]
capacityDict3 = {}
lengthCoefficients = [100, 70]
lengthDict3 = {}
qty1Coefficients = [1, -1.25]
qty1Dict3 = {}
numOfType1Trains = {}
numOfType2Trains = {}
qty2Coefficients = [-1.25, 1]
qty2Dict3 = {}
M = 1000

for composition in range(len(compositions)):
  for c in range(200):
    capacityDict3[(c, composition)] = capacityCoefficients[0] * compositions[composition][0] + capacityCoefficients[1] * compositions[composition][1]
    costDict3[(c, composition)] = costCoefficients[0] * compositions[composition][0] + costCoefficients[1] * compositions[composition][1]
    lengthDict3[(c, composition)] = lengthCoefficients[0] * compositions[composition][0] + lengthCoefficients[1] * compositions[composition][1]
    qty1Dict3[(c, composition)] = qty1Coefficients[0] * compositions[composition][0] + qty1Coefficients[1] * compositions[composition][1]
    qty2Dict3[(c, composition)] = qty2Coefficients[0] * compositions[composition][0] + qty2Coefficients[1] * compositions[composition][1]

print(capacityDict3)

{(0, 0): 420, (1, 0): 420, (2, 0): 420, (3, 0): 420, (4, 0): 420, (5, 0): 420, (6, 0): 420, (7, 0): 420, (8, 0): 420, (9, 0): 420, (10, 0): 420, (11, 0): 420, (12, 0): 420, (13, 0): 420, (14, 0): 420, (15, 0): 420, (16, 0): 420, (17, 0): 420, (18, 0): 420, (19, 0): 420, (20, 0): 420, (21, 0): 420, (22, 0): 420, (23, 0): 420, (24, 0): 420, (25, 0): 420, (26, 0): 420, (27, 0): 420, (28, 0): 420, (29, 0): 420, (30, 0): 420, (31, 0): 420, (32, 0): 420, (33, 0): 420, (34, 0): 420, (35, 0): 420, (36, 0): 420, (37, 0): 420, (38, 0): 420, (39, 0): 420, (40, 0): 420, (41, 0): 420, (42, 0): 420, (43, 0): 420, (44, 0): 420, (45, 0): 420, (46, 0): 420, (47, 0): 420, (48, 0): 420, (49, 0): 420, (50, 0): 420, (51, 0): 420, (52, 0): 420, (53, 0): 420, (54, 0): 420, (55, 0): 420, (56, 0): 420, (57, 0): 420, (58, 0): 420, (59, 0): 420, (60, 0): 420, (61, 0): 420, (62, 0): 420, (63, 0): 420, (64, 0): 420, (65, 0): 420, (66, 0): 420, (67, 0): 420, (68, 0): 420, (69, 0): 420, (70, 0): 420, (71, 0): 420, (

In [ ]:
# Defining Objective Function
m3.setObjective(X2.prod(costDict3))

for c in range(200):
  for d in range(250):
    m3.add

In [ ]:
# Defining Objective Function
m3.setObjective(X2.prod(costDict3))

FirstYConstraint = LinExpr()
SecondYConstraint = LinExpr()

for c in range(200):
  for d in range(250):
    FirstYConstraint.add(X2.prod(capacityDict3, c, '*'))
    SecondYConstraint.add(X2.prod(capacityDict3, c, '*'))
    FirstYConstraint.add(M * Y[c, d])
    SecondYConstraint.add(M * Y[c, d])
    m3.addConstr(FirstYConstraint, GRB.LESS_EQUAL, M + stochasticDemands[c][d])
    m3.addConstr(SecondYConstraint, GRB)

# Generating Constraints
#m3.addConstrs(X2.prod(capacityDict3, c, '*') + M * Y[c, d] <= M + stochasticDemands[c][d] for c in range(200) for d in range(250))
#m3.addConstrs(M * Y[c, d] + X2.prod(capacityDict3, c, '*') >= stochasticDemands[c][d] for c in range(200) for d in range(250))
m3.addConstr(Y.sum() / 50000 < 0.19)
m3.addConstrs(Y.sum(c, '*') / 250 <= 0.5 for c in range(200))
m3.addConstr(X2.prod(qty1Dict3) <= 0)
m3.addConstr(X2.prod(qty2Dict3) <= 0)
m3.addConstrs(X2.prod(lengthDict3, c, '*') <= lengthRequirement[c] for c in range(200))
m3.addConstrs(X2.sum(c, '*') == 1 for c in range(200))

m3.optimize()
m3.dispose()

<ipython-input-7-019273bacdc6>:13: DeprecationWarning: Deprecated, pass a TempConstr or use Model.addLConstr
  m3.addConstr(FirstYConstraint, GRB.LESS_EQUAL, M + stochasticDemands[c][d])


GurobiError: Invalid argument to Model.addConstr